<a href="https://colab.research.google.com/github/Aidan1223f/SERP_2022/blob/main/4thIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

from google.colab import files
uploaded = files.upload()

Saving www-gutenberg-org-files-1342-1342-0-txt.txt to www-gutenberg-org-files-1342-1342-0-txt.txt


In [ ]:
!pip install convokit
from convokit import Corpus, download
corpus = Corpus(filename=download("switchboard-corpus"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 161, in _make_candidate_from_dist
    base = self._installed_candidate_cache[dist.canonical_name]
KeyError: 'clean-text'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/commands/install.py", line 339, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 94, in resol

In [ ]:
corpus.print_summary_stats()

print(corpus)


Number of Speakers: 440
Number of Utterances: 122646
Number of Conversations: 1155


Preproccessing::

In [ ]:
file = open("www-gutenberg-org-files-1342-1342-0-txt.txt", "r", encoding = "utf8")


line = []
for i in file:
    line.append(i)


txtData = ""
for i in line:
  txtData = ' '. join(line) 

txtData = txtData.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

 
txtData = txtData.split()
txtData = ' '.join(txtData)
print(txtData[:300])

print(len(txtData))

The Project Gutenberg eBook of Pride and prejudice, by Jane Austen This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gute
104345


Tokenizer:

In [ ]:
token = Tokenizer(oov_token='<OOV>')
token.fit_on_texts([txtData])

# saving the tokenizer for predict function
pickle.dump(token, open('token.pkl', 'wb'))

seq_data = token.texts_to_sequences([txtData])[0]
print(seq_data[:30])
print(len(seq_data))

[2, 646, 501, 375, 3, 138, 5, 307, 27, 67, 116, 39, 375, 14, 22, 2, 427, 3, 871, 1347, 7, 2, 502, 872, 5, 86, 99, 873, 3, 2]
18732


In [ ]:
vocabSize =  len(token.word_index) + 1
print(vocabSize)

3043


Creates sequence of 3 words and predicted word 

In [ ]:
seq = []
for i in range(4, len(seq_data)): 
  wrd = seq_data[i-4 : i + 1]
  seq.append(wrd)

print("sequence length: " , len(seq))
seq = np.array(seq)
print(seq[:15])

sequence length:  18728
[[  2 646 501 375   3]
 [646 501 375   3 138]
 [501 375   3 138   5]
 [375   3 138   5 307]
 [  3 138   5 307  27]
 [138   5 307  27  67]
 [  5 307  27  67 116]
 [307  27  67 116  39]
 [ 27  67 116  39 375]
 [ 67 116  39 375  14]
 [116  39 375  14  22]
 [ 39 375  14  22   2]
 [375  14  22   2 427]
 [ 14  22   2 427   3]
 [ 22   2 427   3 871]]


Data represents the first 3 letters that the alg will be reading 
:  Predict represents predicted word

In [ ]:
data = []
predict = []

for i in seq:
    data.append(i[0:4])
    predict.append(i[4])
    
data = np.array(data)
predict = np.array(predict)

print(data)
print("\n")
print(predict)


[[   2  646  501  375]
 [ 646  501  375    3]
 [ 501  375    3  138]
 ...
 [3042   45  178  234]
 [  45  178  234   84]
 [ 178  234   84   79]]


[   3  138    5 ...   84   79 1089]




converts the class vectors into binary class matrix




In [ ]:
predict = to_categorical(predict, num_classes=vocabSize)
print(predict[:5])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Model Creation: LSTM 

In [ ]:
model = Sequential()
model.add(Embedding(vocabSize, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocabSize, activation="softmax"))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             30430     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 3043)              3046043   
                                                                 
Total params: 16,125,473
Trainable params: 16,125,473
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(data, predict, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70


ValueError: ignored

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  txt = input("Enter the first 3 words:  ")
  if txt =="0":
    print("complete")
  else: 
    try: 
      txt = txt.split(" ")
      txt = txt[-4:]
      print(txt)
      Predict_Next_Words(model, tokenizer, txt)

    except Exception as e:
      print("ERROR", e)
      continue


Enter the first 3 words:  where are you 
['are', 'you', '']
1/1 [==============================] - 1s 1s/step
the
Enter the first 3 words:  what do you 
['do', 'you', '']
1/1 [==============================] - 0s 38ms/step
the
Enter the first 3 words:  I am going to 
['going', 'to', '']
1/1 [==============================] - 0s 36ms/step
the
Enter the first 3 words:  I want to eat 
['to', 'eat', '']
1/1 [==============================] - 0s 34ms/step
the
Enter the first 3 words:  where did you 
['did', 'you', '']
1/1 [==============================] - 0s 37ms/step
the
Enter the first 3 words:  what did you 
['did', 'you', '']
1/1 [==============================] - 0s 32ms/step
the


KeyboardInterrupt: ignored

In [ ]:
# text input testing

testList = ["where are you", "what do you", "I am going to", "I want to eat", "where did you ", "what did you", ""  ]